In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold

from sklearn.model_selection import KFold
from sklearn import metrics

In [2]:
data = pd.read_csv(r"data_no_corr_70.csv")

data_defects = data.loc[data['defects'] == True]

data_no_defects = data.loc[data['defects'] == False]
data_no_defects = data_no_defects.sample(n=len(data_defects))

print(len(data_no_defects))
print(len(data_defects))

balanced_data = data_defects.append(data_no_defects)
balanced_data = balanced_data.sample(frac = 1).reset_index(drop = True)
balanced_data

2665
2665


,lineCount,cyclomaticComplexity,essentialComplexity,designComplexity,operatorsOperands,volume,programLength,difficulty,intelligence,effort,...,hlineCount,hlineComments,hlineBlanks,hCodeAndComment,uniqueOperators,uniqueOperands,totalOperators,totalOperands,branchCount,defects
0,8.0,1.0,1.0,1.0,13.0,39.00,0.56,1.80,21.67,70.20,...,4,1,1,0,3.0,5.0,7.0,6.0,1.0,False
1,138.0,35.0,21.0,17.0,498.0,3191.88,0.02,51.27,62.26,163640.68,...,115,0,21,0,26.0,46.0,210.0,147.0,51.0,True
2,121.0,7.0,1.0,7.0,453.0,2871.95,0.04,22.31,128.71,64080.43,...,73,20,24,0,17.0,64.0,285.0,168.0,13.0,True
3,7.0,1.0,1.0,1.0,14.0,48.43,0.28,3.60,13.45,174.36,...,4,0,1,0,6.0,5.0,8.0,6.0,1.0,False
4,108.0,19.0,10.0,11.0,262.0,1514.72,0.04,25.05,60.46,37947.63,...,68,21,17,0,17.0,38.0,150.0,112.0,37.0,True
5,18.0,3.0,1.0,3.0,46.0,205.13,0.11,9.00,22.79,1846.20,...,15,0,1,0,11.0,11.0,28.0,18.0,5.0,True
6,13.0,2.0,1.0,2.0,22.0,76.11,0.19,5.40,14.09,410.98,...,11,0,0,0,6.0,5.0,13.0,9.0,3.0,False
7,5.0,1.0,1.0,1.0,17.0,60.94,0.29,3.50,17.41,213.31,...,3,0,0,0,6.0,6.0,10.0,7.0,1.0,False
8,14.0,3.0,1.0,2.0,42.0,187.30,0.13,7.80,24.01,1460.91,...,10,0,2,0,12.0,10.0,29.0,13.0,5.0,True
9,5.0,2.0,1.0,2.0,21.0,84.00,0.13,7.50,11.20,630.00,...,5,0,0,1,10.0,6.0,12.0,9.0,3.0,False


In [3]:
features = list(balanced_data.columns.values)
features.remove('defects')

In [4]:
train = balanced_data.sample(frac = 0.8, random_state = 1)
test = balanced_data.loc[~balanced_data.index.isin(train.index)]

train_data_array=train.as_matrix(columns = features)
train_class_array= train['defects'].values
test_data_array=test.as_matrix(columns = features)
test_class_array= test['defects'].values

In [5]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

kf=KFold(n_splits=10, random_state=None, shuffle=False)
validation_score=0
for train_index, test_index in kf.split(train_data_array):
    X_train, X_test = train_data_array[train_index], train_data_array[test_index]
    y_train, y_test = train_class_array[train_index], train_class_array[test_index]
    clf.fit(X_train, y_train)
    validation_pred=clf.predict(X_test)
    validation_score+=metrics.accuracy_score(y_test,validation_pred)

print("Validation Accuracy:   %0.3f" % (validation_score/kf.get_n_splits()))

pred = clf.predict(test_data_array)
score = metrics.accuracy_score(test_class_array, pred)
print("accuracy:   %0.3f" % score)

score = metrics.precision_score(test_class_array, pred)
print("Precision:   %0.3f" % score)

score = metrics.recall_score(test_class_array, pred)
print("Recall:   %0.3f" % score)

score = metrics.f1_score(test_class_array, pred)
print("F-measure:   %0.3f" % score)

Validation Accuracy:   0.526
accuracy:   0.527
Precision:   0.698
Recall:   0.070
F-measure:   0.128


In [6]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)

clf.fit(train_data_array, train_class_array)

pred = clf.predict(test_data_array)
score = metrics.accuracy_score(test_class_array, pred)
print("accuracy:   %0.3f" % score)

score = metrics.precision_score(test_class_array, pred)
print("Precision:   %0.3f" % score)

score = metrics.recall_score(test_class_array, pred)
print("Recall:   %0.3f" % score)

score = metrics.f1_score(test_class_array, pred)
print("F-measure:   %0.3f" % score)

accuracy:   0.699
Precision:   0.702
Recall:   0.676
F-measure:   0.689


In [7]:
from sklearn import svm

clf = svm.SVC()

clf.fit(train_data_array, train_class_array)

pred = clf.predict(test_data_array)
score = metrics.accuracy_score(test_class_array, pred)
print("accuracy:   %0.3f" % score)

score = metrics.precision_score(test_class_array, pred)
print("Precision:   %0.3f" % score)

score = metrics.recall_score(test_class_array, pred)
print("Recall:   %0.3f" % score)

score = metrics.f1_score(test_class_array, pred)
print("F-measure:   %0.3f" % score)

accuracy:   0.615
Precision:   0.567
Recall:   0.922
F-measure:   0.702
